In [ ]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np
from utils import * 

In [ ]:
# Define the RNN model
class RNN(nn.Module):
    def __init__(self, vocabulary_size, num_labels, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Args:
        vocabulary_size (int): The size of the vocabulary (number of unique tokens in the input text).
        num_labels (int): The number of output classes (labels).
        embedding_dim (int): The dimension of the word embeddings (the vector size representing each word).
        hidden_dim (int): The number of units in the hidden state of the RNN.
        n_layers (int): The number of RNN layers to stack.
        dropout (float, optional): The probability for dropout regularization (default is 0.5).
        """
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, num_labels)

    def forward(self, x):
        embedded = self.embedding(x)
        out, hidden = self.rnn(embedded)
        out = self.dropout(out)
        out = self.fc(out[:, -1])
        return out

In [ ]:
# Ensure GPU or CPU compatibility
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 32

train_data = ...  
train_labels = ... 
val_data = ...
val_loader = ...


train_data = torch.tensor(train_data, dtype=torch.long)
train_labels = torch.tensor(train_labels, dtype=torch.long)
val_data = torch.tensor(val_data, dtype=torch.long)
val_labels = torch.tensor(val_labels, dtype=torch.long)

# Create TensorDataset and DataLoader
train_dataset = TensorDataset(train_data, train_labels)
val_dataset = TensorDataset(val_data, val_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
epochs = 20
param_grid = {
    'embedding_dim': [100, 200, 400],
    'hidden_dim': [128, 256, 512],
    'n_layers': [1, 2],
    'learning_rate': [0.001, 0.0001]
}

best_params, best_accuracy = tune_model_hyperparameters(train_data, train_labels, param_grid, epochs)
print(f"Best Parameters: {best_params}")
print(f"Best Accuracy: {best_accuracy}")

embedding_dim, hidden_dim, n_layers, learning_rate = best_params

In [ ]:
num_labels = 6  # Ekman's six emotions

vocabulary_size = len(train_data[0]) + 1 

rnn_model = RNN(vocabulary_size, num_labels, embedding_dim, hidden_dim, n_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=learning_rate)

In [ ]:
print("RNN Start Training:\n")
train_losses, val_losses, val_accs = train_and_validate(
    rnn_model, optimizer, criterion, train_loader, val_loader, epochs, device
)
plot_losses("RNN", train_losses, val_losses, epochs)